# Day 4

## Getting Started

Navigate to [Advent of Code Day 4](https://adventofcode.com/2025/day/4)

Save the problem input (I have saved as a text file called `AOC25_4_in.txt`)

## Understanding the problem

In part 1, we are faced with a fairly simple task: count neighbouring cells which contain rolls of paper. Let's look a a simple concept for iterate of neighbour cells. Suppose we call a cell $[x, y]$, meaning that it is in row $x$ and column $y$.

Since Python works with 0-indexed arrays, $0 <= x < n$ and $0 <= y < m$, where $n$ is the number of rows in the grid and $y$ is the number of columns.

To examine neighbouring cells, we consider a shift $[dx, dy]$, such that $-1 <= dx <= 1$ and $-1 <= dy <= 1$, where at least one of $dx$ or $dy$ must be non-zero (otherwise it is the cell itself, not the neighbour). Then for cell $[x, y]$ we well examine all cells $[x + dx, y + dy]$ subject to the above constraints.

Note also that a neighbouring cell must be in the grid itself, so we impose two additional constraints: $0 <= x + dx < n$ and $0 <= y + dy < m$.

With this constraints, it is trivial to count neighbouring cells with rolls.

In part 2, the problem is much more involved. Naively we might thing "let's just iterate over the grid as many times as we need to" - but this could be very slow! There are 20,000 cells, and if we consider a scenario where we need to iterate over the grid from the outside to the centre to remove all rolls, this count take 10,000 iterations. Already we're talking about more than $10^8$ operations, and with a high constant factor for each iteration, we're likely looking at an even higher order of magnitude.

Luckily, there is a way we can avoid doing all those iterations: [**breadth-first search (BFS)**](https://en.wikipedia.org/wiki/Breadth-first_search). This is a standard efficient graph traversal algorithm which looks at cells ('nodes', in graph terminology) as they become of interest to us, and then does not need to reconsider them again.

Let us consider our grid as follows:

- each cell $[x, dx]$ is a 'node' (or vertex) of a graph
- each neighbour of that cell $[x + dx, y + dy]$ is connected to the $[x, y]$ by an edge

BFS works as follows:

- find the first cells we are interested in (the ones with rolls with fewer than 4 neighbouring rolls), and add them to a **queue**
- process all the cells in the queue one at a time:
    - each time we process a cell, we examine its neighbours to see if we have any new cells that need to be added to the queue - this will be the case if they now have fewer than 4 neighbours
    - we continue until there are now more cells in the queue, counting every cell that we remove a roll from

And that's it! Simple. BFS saves the day.

## Part 1 - counting only

Read in the inputs (see day 1 for explanation):

In [1]:
# open the file
import sys
read = sys.stdin.read
f = open("AOC25_4_in.txt")

# create a proper matrix structure for the grid
rolls = [list(x) for x in f.read().split('\n')]

Define a function to count neighbouring cells with rolls:

In [2]:
def count_neighbours(x, y):
    tot = 0
    
    # a neighbour can be -1, 0 or 1 away in the x direction
    for dx in range(-1, 2):
    
        # if the x-shift takes us out of range (because we were at the edge of the grid), we do not consider this value
        if x + dx < 0 or x + dx >= len(rolls):
            continue
        
        # a neighbour can be -1, 0 or 1 away in the y direction
        for dy in range(-1, 2):
            
            # if the y-shift takes us out of range (because we were at the edge of the grid), we do not consider this value
            if y + dy < 0 or y + dy >= len(rolls[x]):
                continue
            
            # if there is no shift at all, it is not a neighbour, it is the cell itself, so ignore
            if dx == dy == 0:
                continue
            
            # if the neighbouring cell contains '@', then it is a roll
            if rolls[x + dx][y + dy] == '@':
                tot += 1
    return tot

Iterate over all cells, and then return the answer:

In [3]:
# create answer variable
ans = 0

# iterate over all cells in the grid
for i in range(len(rolls)):
    for j in range(len(rolls[i])):
        
        # using count_neighbours function, add 1 to our total if there are fewer than 4 neighbouring rolls
        if rolls[i][j] == '@' and count_neighbours(i, j) < 4:
            ans += 1

# print answer
print(ans)


1491


And that's all that part 1 requires!

## Part 2 - removing iteratively

For our breadth-first search (BFS), I will introduce a new function that retrieves all neighbouring cells with a roll-count of less than 4.

In [4]:
# define a function to retrieve all neighbouring rolls which are next to fewer than 4 rolls themselves
def get_neighbouring_rolls(x, y):
    
    # create an array to store the valid rolls
    valid_neighbours = []
    
    # use the same shift logic as in count_neighbours
    for dx in range(-1, 2):
        if x + dx < 0 or x + dx >= len(rolls):
            continue
        for dy in range(-1, 2):
            if y + dy < 0 or y + dy >= len(rolls[x]):
                continue
            if dx == dy == 0:
                continue
            
            # this time we're checking whether the cell contains '@' AND whether it has fewer than 4 neighbouring rolls itself
            # if so we add it to our valid neighbouring rolls array
            if rolls[x + dx][y + dy] == '@' and count_neighbours(x + dx, y + dy) < 4:
                valid_neighbours.append((x + dx, y + dy))
    
    return valid_neighbours

I then create my queue, using a Python data structure called [`deque`](https://en.wikipedia.org/wiki/Double-ended_queue) from the `collections` library. The reason for using this is the way `deque` assigns values in memory - it is much more efficient than a standard array (`list`), and allows us to efficiently remove the first element of the queue once we've finished processing it. First, I will need to import it:

In [5]:
from collections import deque

Then I'll set up my queue `Q` by adding all the cells that **currently** have a roll and fewer than 4 neighbouring rolls.

In [6]:
ans = 0

# for BFS, we will use a queue structure called a deque, which allows us to efficiently remove the first element
# note that a regular array is much less efficient at doing this, due to the data structure type and the way it allocates values into memory
Q = deque()

# iterate over all cells in the grid
for i in range(len(rolls)):
    for j in range(len(rolls[i])):
        
        # using count_neighbours function, add cells to our queue if they are a roll with fewer than 4 neighbours
        if rolls[i][j] == '@' and count_neighbours(i, j) < 4:
            Q.append((i, j))


Finally, I'll run my BFS algorithm as described above, and return the answer:

In [7]:
# this is the BFS: while there remain rolls in our queue for removal, we continue
while Q:
    # take the front roll in the queue
    i, j = Q.popleft()
    
    # if we've processed this roll already, we don't need to do it again
    if rolls[i][j] == '.':
        continue
    
    # if we haven't processed already, we add one to our count of removed rolls
    ans += 1
    
    # we then set this cell to '.' to reflect the fact there is no longer a roll there
    rolls[i][j] = '.'
    
    # finally, we re-examine its neighbours, and add any valid ones back to our queue
    for x, y in get_neighbouring_rolls(i, j):
        Q.append((x, y))

print(ans)

8722


And day 4 is complete! This ran in Python in around 0.07 seconds, which is a perfectly acceptable runtime. Some small improvements could be made (like ensuring that a cell is only adding to `Q` once), but the improvement factor would be fairly negligible here, as we can simply disregard the cell any additional times it is added (and even so this can be necessary no more than 4 times - as the last 4 of its neighbouring cells are processed).